In [27]:
pip install creme


In [28]:
import math
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
import creme

In [29]:
docs=[('Chinese Beijing chinese', 'yes'),
('Chinese chinese shanghai', 'yes'),
('Chinese india Maco','yes'),
('Tokyo Japan Chinese','no')]

In [30]:
corpus = ['Ths is the firdt document.',
          'This document is the second document',
          'And this the third one.',
          'Is this the first document?',
          ]
bow = creme.feature_extraction.TFIDF()
for sentence in corpus:
      print(bow.transform_one(sentence))

{'ths': 0.4472135954999579, 'is': 0.4472135954999579, 'the': 0.4472135954999579, 'firdt': 0.4472135954999579, 'document': 0.4472135954999579}
{'this': 0.35355339059327373, 'document': 0.7071067811865475, 'is': 0.35355339059327373, 'the': 0.35355339059327373, 'second': 0.35355339059327373}
{'and': 0.4472135954999579, 'this': 0.4472135954999579, 'the': 0.4472135954999579, 'third': 0.4472135954999579, 'one': 0.4472135954999579}
{'is': 0.4472135954999579, 'this': 0.4472135954999579, 'the': 0.4472135954999579, 'first': 0.4472135954999579, 'document': 0.4472135954999579}


In [31]:
model = compose.Pipeline(
    ('tokenize', feature_extraction.BagOfWords(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1)))

In [32]:
%%time
for sentence, label in docs:
     model = model.fit_one(sentence, label)

CPU times: user 139 µs, sys: 14 µs, total: 153 µs
Wall time: 159 µs


In [33]:
new_unseen_text = 'Tokyo india'

In [34]:
model.predict_one(new_unseen_text)

'yes'

In [35]:
model.fit_one('India USA','may be')

Pipeline (
  BagOfWords (
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x78e91ac52f80>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)

In [36]:
model.predict_one("India Germany")

'may be'

In [37]:
from creme import compose
from creme import datasets
from creme import linear_model
from creme import metrics
from creme import preprocessing

X_y = datasets.Phishing()

model = compose.Pipeline(
   preprocessing.StandardScaler(),
 linear_model.LogisticRegression()
)
metric = metrics.Accuracy()


for x, y in X_y:
    y_pred = model.predict_one(x)      # make a prediction
    metric = metric.update(y, y_pred)  # update the metric
    model = model.fit_one(x, y)        # make the model learn

metric

Accuracy: 89.20%

In [38]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SMSSpamCollection.txt',
                 sep='\t',
                 header=None,
                 names=['label', 'message'])

# To check if it worked
print(df.head())


  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [39]:
df.shape

(5572, 2)

In [40]:
from sklearn.model_selection import train_test_split
df_train,df_test=train_test_split(df)

In [41]:
df_train

,label,message
4800,ham,I dont know why she.s not getting your messages
775,ham,Just sleeping..and surfing
3366,ham,"Sorry, I'll call later"
2611,ham,"As usual..iam fine, happy &amp; doing well..:)"
2106,ham,I fetch yun or u fetch?
...,...,...
850,spam,Today's Offer! Claim ur £150 worth of discount...
5054,ham,Lmao you know me so well...
4646,ham,Are you planning to come chennai?
4250,ham,Loosu go to hospital. De dont let it careless.


In [42]:
#Convert dataframe to list of tuples
df_train=df_train.to_records(index=False)
df_test=df_test.to_records(index=False)

In [43]:
df_train

rec.array([('ham', 'I dont know why she.s not getting your messages'),
           ('ham', 'Just sleeping..and surfing'),
           ('ham', "Sorry, I'll call later"), ...,
           ('ham', 'Are you planning to come chennai?'),
           ('ham', 'Loosu go to hospital. De dont let it careless.'),
           ('spam', 'Loans for any purpose even if you have Bad Credit! Tenants Welcome. Call NoWorriesLoans.com on 08717111821')],
          dtype=[('label', 'O'), ('message', 'O')])

In [44]:
#Creatingthe pipeline
#1st function is creating the TFIDF
#2nd function is the naive bayes predictor
import math
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
import creme
model=compose.Pipeline(
    ('tokenize',feature_extraction.BagOfWords(lowercase=False)),
    ('nb',naive_bayes.MultinomialNB(alpha=1))
)

In [45]:
from creme import metrics
metric=metrics.Accuracy()
#Traning the model row by row
for label,sentence in df_train:
  model=model.fit_one(sentence, label)
  y_pred=model.predict_one(sentence)
  metric=metric.update(label, y_pred)

In [46]:
## Traning Data Accuracy
metric

Accuracy: 99.31%

In [47]:
##Test Data Accuracy
test_metric=metrics.Accuracy()
for label,sentence in df_test:
  y_pred=model.predict_one(sentence)
  test_metric=test_metric.update(label, y_pred)

In [48]:
##Test Metric
test_metric

Accuracy: 97.63%

In [49]:
model.fit_one("This guy is neutral","ham")
model.fit_one("Everybody is neutral","ham")

Pipeline (
  BagOfWords (
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x78e91ac52f80>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)